In [1]:
import pandas as pd
import numpy as np
import pickle as p

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
#tf.enable_eager_execution()
from sklearn.model_selection import train_test_split

from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

Compile the training data

In [2]:
meta_data = pd.read_csv('/home/ec2-user/training_data_meta.csv')

#Join cleaned class data with training metadata

class_data = pd.read_csv('/home/ec2-user/stanford_labels_cleaned.csv')

meta_data = pd.merge(meta_data, class_data, on = 'class', how = 'left')

meta_data = meta_data.loc[meta_data['Body Type'].isin(['Coupe', 'Sedan'])].copy()

meta_data['is_sedan_target'] = (meta_data['Body Type'] == 'Sedan').astype(int)

image_dict = p.load(open('/home/ec2-user/scaled_bounded_grayscale_dict.p', 'rb'))

training_data = []
for i in meta_data[['is_sedan_target', 'fname']].iterrows():
    row = [i[1]['is_sedan_target']]
    row.extend(image_dict[i[1]['fname']].flatten())
    training_data.append(row)

training_data = np.array(training_data).astype(float)

Prep the data

In [3]:
X_train, X_test, y_train, y_test = train_test_split(training_data[:,1:], 
                                                    training_data[:,0], 
                                                    test_size=0.30, 
                                                    random_state=420)

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

negatives = []
for ind, elem in enumerate(y_train):
    if elem == 0:
        negatives.append(ind)
np.random.shuffle(negatives)

neg_dupe_target = 2 * y_train.sum() - y_train.shape[0]

X_train = np.concatenate([X_train, X_train[negatives[0:neg_dupe_target],:]],
                         axis = 0
                        )
y_train = np.concatenate([y_train, y_train[negatives[0:neg_dupe_target]]],
                         axis = 0
                        )

y_train = np.concatenate([1 - y_train.reshape(-1,1), y_train.reshape(-1,1)], axis = 1)
y_test = np.concatenate([1 - y_test.reshape(-1,1), y_test.reshape(-1,1)], axis = 1)

Define the model

In [4]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    if type(features) is dict:
        features = features['input']
    #Define model architecture
    
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    input_layer = tf.reshape(features, [-1, 200, 200, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 3x3 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 200, 200, 1]
    # Output Tensor Shape: [batch_size, 200, 200, 32]
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 200, 200, 32]
    # Output Tensor Shape: [batch_size, 100, 100, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 3x3 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 100, 100, 32]
    # Output Tensor Shape: [batch_size, 100, 100, 64]
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 100, 100, 64]
    # Output Tensor Shape: [batch_size, 50, 50, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #3
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 50, 50, 64]
    # Output Tensor Shape: [batch_size, 25, 25, 64]
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 25, 25, 64]
    # Output Tensor Shape: [batch_size, 25 * 25 * 64]
    pool3_flat = tf.reshape(pool3, [-1, 25 * 25 * 64])

    # Dense Layer
    # Densely connected layer with 256 neurons
    # Input Tensor Shape: [batch_size, 25 * 25 * 64]
    # Output Tensor Shape: [batch_size, 256]
    dense = tf.layers.dense(inputs=pool3_flat, units=64, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=2)
    
    #Predict Op
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            # Generate predictions (for PREDICT and EVAL mode)
            "classes": tf.argmax(input=logits, axis=1),
            # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
            # `logging_hook`.
            "probabilities": tf.nn.softmax(logits, name="softmax_tensor"),
            "logits" : logits
            
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #Define model outputs
        # Calculate Loss (for both TRAIN and EVAL modes)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels[:,1], logits=logits)
        
        logging_hook = tf.train.LoggingTensorHook(
            {"loss" : loss,
            },
            every_n_iter=10)
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, train_op=train_op, training_hooks = [logging_hook])
    # Add evaluation metrics (for EVAL mode)
    if mode == tf.estimator.ModeKeys.EVAL:
        #Define model outputs
        # Calculate Loss (for both TRAIN and EVAL modes)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels[:,1], logits=logits)
        
        predictions = {
          # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
          # `logging_hook`.
          "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }
        #swap this for AUROC?
        eval_metric_ops = {
          "eval_accuracy": tf.metrics.auc(
              labels=labels, predictions=predictions["probabilities"])}

        return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [5]:
session_config = tf.ConfigProto(
      inter_op_parallelism_threads=16,
      intra_op_parallelism_threads=16,
      )

config = tf.estimator.RunConfig(session_config = session_config,
                                log_step_count_steps = 10,
                                save_summary_steps = 10
                               )

# Create the Estimator
car_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="/home/ec2-user/convnet_model", config = config)

INFO:tensorflow:Using config: {'_model_dir': '/home/ec2-user/convnet_model', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': intra_op_parallelism_threads: 16
inter_op_parallelism_threads: 16
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f97cdfacf28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
#custom input function using training_image_processor with tf.data.Dataset
def image_processor_train_input_fn(image_processor, X_train, y_train, batch_size):
    
    dataset = tf.data.Dataset.from_generator(
        generator = lambda : image_processor.flow(X_train.reshape((X_train.shape[0], 200, 200, 1)), 
                                                  y_train, batch_size, shuffle = True), 
        output_types = (np.float32, np.int32))
    
    dataset = dataset.shuffle(512).repeat()
    
    return dataset

#Shearing, no zooming
training_image_processor = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./255,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range=15,
    horizontal_flip=True,
    data_format = 'channels_last',
)

def image_processor_eval_input_fn(X_test, y_test, batch_size):
    
    dataset = tf.data.Dataset.from_tensor_slices((X_test.astype('float32') / 255.0, 
                                                  y_test.astype('int32')))
        
    return dataset.batch(batch_size)

Train

In [21]:
car_classifier.train(
      input_fn=lambda:image_processor_train_input_fn(training_image_processor, X_train, y_train, 128),
      steps=2000
    )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-1869
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1869 into /home/ec2-user/convnet_model/model.ckpt.
INFO:tensorflow:loss = 0.48121965, step = 1870
INFO:tensorflow:loss = 0.48121965
INFO:tensorflow:global_step/sec: 0.254718
INFO:tensorflow:loss = 0.43163192, step = 1880 (39.261 sec)
INFO:tensorflow:loss = 0.43163192 (39.260 sec)
INFO:tensorflow:global_step/sec: 0.297032
INFO:tensorflow:loss = 0.4936751, step = 1890 (33.666 sec)
INFO:tensorflow:loss = 0.4936751 (33.666 sec)
INFO:tensorflow:global_step/sec: 0.312918
INFO:tensorflow:loss = 0.5973296, step = 1900 (31.957 sec)
INFO:tensorflow:loss = 0.5973296 (31.957 sec)
INFO:tensorflow:global_step/sec: 0.314807
INFO:tensorflow:loss

INFO:tensorflow:loss = 0.44729656, step = 2380 (33.564 sec)
INFO:tensorflow:loss = 0.44729656 (33.564 sec)
INFO:tensorflow:global_step/sec: 0.298663
INFO:tensorflow:loss = 0.4427994, step = 2390 (33.483 sec)
INFO:tensorflow:loss = 0.4427994 (33.483 sec)
INFO:tensorflow:global_step/sec: 0.299641
INFO:tensorflow:loss = 0.51925415, step = 2400 (33.373 sec)
INFO:tensorflow:loss = 0.51925415 (33.373 sec)
INFO:tensorflow:global_step/sec: 0.298898
INFO:tensorflow:loss = 0.48078972, step = 2410 (33.456 sec)
INFO:tensorflow:loss = 0.48078972 (33.456 sec)
INFO:tensorflow:global_step/sec: 0.299553
INFO:tensorflow:loss = 0.41276345, step = 2420 (33.383 sec)
INFO:tensorflow:loss = 0.41276345 (33.383 sec)
INFO:tensorflow:global_step/sec: 0.298963
INFO:tensorflow:loss = 0.35167378, step = 2430 (33.449 sec)
INFO:tensorflow:loss = 0.35167378 (33.449 sec)
INFO:tensorflow:global_step/sec: 0.311526
INFO:tensorflow:loss = 0.4929119, step = 2440 (32.100 sec)
INFO:tensorflow:loss = 0.4929119 (32.100 sec)
INF

INFO:tensorflow:global_step/sec: 0.313014
INFO:tensorflow:loss = 0.41073245, step = 2950 (31.948 sec)
INFO:tensorflow:loss = 0.41073245 (31.948 sec)
INFO:tensorflow:global_step/sec: 0.302025
INFO:tensorflow:loss = 0.49561173, step = 2960 (33.110 sec)
INFO:tensorflow:loss = 0.49561173 (33.110 sec)
INFO:tensorflow:global_step/sec: 0.312183
INFO:tensorflow:loss = 0.42855123, step = 2970 (32.033 sec)
INFO:tensorflow:loss = 0.42855123 (32.033 sec)
INFO:tensorflow:global_step/sec: 0.310865
INFO:tensorflow:loss = 0.4126223, step = 2980 (32.168 sec)
INFO:tensorflow:loss = 0.4126223 (32.168 sec)
INFO:tensorflow:global_step/sec: 0.299329
INFO:tensorflow:loss = 0.3738147, step = 2990 (33.408 sec)
INFO:tensorflow:loss = 0.3738147 (33.408 sec)
INFO:tensorflow:global_step/sec: 0.312255
INFO:tensorflow:loss = 0.39612848, step = 3000 (32.025 sec)
INFO:tensorflow:loss = 0.39612848 (32.025 sec)
INFO:tensorflow:global_step/sec: 0.300141
INFO:tensorflow:loss = 0.39324036, step = 3010 (33.318 sec)
INFO:ten

INFO:tensorflow:loss = 0.29591656 (31.890 sec)
INFO:tensorflow:global_step/sec: 0.324016
INFO:tensorflow:loss = 0.41201934, step = 3490 (30.863 sec)
INFO:tensorflow:loss = 0.41201934 (30.863 sec)
INFO:tensorflow:global_step/sec: 0.313286
INFO:tensorflow:loss = 0.40474916, step = 3500 (31.920 sec)
INFO:tensorflow:loss = 0.40474916 (31.919 sec)
INFO:tensorflow:global_step/sec: 0.313547
INFO:tensorflow:loss = 0.38711894, step = 3510 (31.893 sec)
INFO:tensorflow:loss = 0.38711894 (31.893 sec)
INFO:tensorflow:global_step/sec: 0.32642
INFO:tensorflow:loss = 0.4935946, step = 3520 (30.635 sec)
INFO:tensorflow:loss = 0.4935946 (30.635 sec)
INFO:tensorflow:global_step/sec: 0.29979
INFO:tensorflow:loss = 0.3504209, step = 3530 (33.357 sec)
INFO:tensorflow:loss = 0.3504209 (33.357 sec)
INFO:tensorflow:global_step/sec: 0.299753
INFO:tensorflow:loss = 0.5137768, step = 3540 (33.361 sec)
INFO:tensorflow:loss = 0.5137768 (33.361 sec)
INFO:tensorflow:global_step/sec: 0.301924
INFO:tensorflow:loss = 0.

Evaluate Performance

In [22]:
print('training error')
eval_results = car_classifier.evaluate(
    input_fn=lambda:image_processor_eval_input_fn(X_train, y_train, 32))
print(eval_results)

print('testing error')
eval_results = car_classifier.evaluate(
    input_fn=lambda:image_processor_eval_input_fn(X_test, y_test, 32))
print(eval_results)

training error
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-21T05:16:32Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-3869
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-21-05:17:23
INFO:tensorflow:Saving dict for global step 3869: eval_accuracy = 0.95940447, global_step = 3869, loss = 0.25836483
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3869: /home/ec2-user/convnet_model/model.ckpt-3869
{'eval_accuracy': 0.95940447, 'loss': 0.25836483, 'global_step': 3869}
testing error
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-21T05:17:24Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-3869
INFO:tensorf

Run inference

In [9]:
predicted_classes = car_classifier.predict(
    input_fn=lambda:image_processor_eval_input_fn(X_test, y_test, 32))

testing_analysis = []

for pred, label in zip(predicted_classes, y_test):
    hold = [pred['probabilities'][1], pred['classes'], label[1], pred['logits'][0], pred['logits'][1]]
    testing_analysis.append(hold)
    
testing_analysis = pd.DataFrame(testing_analysis, columns = ['prob', 'class', 'label', 'logit_0', 'logit_1'])
testing_analysis['logit_1_decile'] = pd.qcut(testing_analysis.logit_1, 10, labels = False)
testing_analysis['logit_0_decile'] = pd.qcut(testing_analysis.logit_0, 10, labels = False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-4024
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [10]:
testing_analysis.groupby(['logit_1_decile', 'label'])['class'].value_counts()

logit_1_decile  label  class
0               0      0        104
                1      0          5
1               0      0         91
                1      0         17
2               0      0         74
                1      0         35
3               0      0         67
                       1          2
                1      0         39
4               0      1         27
                       0         24
                1      1         38
                       0         20
5               0      1         27
                1      1         81
6               0      1         25
                1      1         83
7               0      1          6
                1      1        103
8               0      1          8
                1      1        100
9               0      1          1
                1      1        108
Name: class, dtype: int64

In [11]:
testing_analysis.groupby(['logit_0_decile', 'label'])['class'].value_counts()

logit_0_decile  label  class
0               0      1          1
                1      1        108
1               0      1          9
                1      1         99
2               0      1          8
                1      1        101
3               0      1         19
                1      1         89
4               0      1         29
                       0          1
                1      1         78
                       0          1
5               0      1         28
                       0         23
                1      1         37
                       0         20
6               0      0         62
                       1          2
                1      0         43
                       1          1
7               0      0         82
                1      0         27
8               0      0         89
                1      0         19
9               0      0        103
                1      0          6
Name: class, dtype: int64

In [12]:
testing_analysis.groupby('label')['class'].value_counts()

label  class
0      0        360
       1         96
1      1        513
       0        116
Name: class, dtype: int64

In [ ]:
for i in testing_analysis.loc[testing_analysis.logit_1_decile == 5].index[0:20]:
    plt.figure()
    row = testing_analysis.iloc[i]
    plt.title('class %f label %f'%(row['class'], row['label']))
    imshow(Image.fromarray((X_test[i,:].reshape((200,200))).astype('uint8'), mode = 'L'))

SUVs and Verts

In [25]:
meta_data = pd.read_csv('/home/ec2-user/training_data_meta.csv')

#Join cleaned class data with training metadata

class_data = pd.read_csv('/home/ec2-user/stanford_labels_cleaned.csv')

meta_data = pd.merge(meta_data, class_data, on = 'class', how = 'left')

meta_data = meta_data.loc[meta_data['Body Type'].isin(['SUV', 'Convertible'
                                                      ])].copy()

meta_data['is_sedan_target'] = (meta_data['Body Type'] == 'SUV').astype(int)

image_dict = p.load(open('/home/ec2-user/scaled_bounded_grayscale_dict.p', 'rb'))

oot_data = []
for i in meta_data[['is_sedan_target', 'fname']].iterrows():
    row = [i[1]['is_sedan_target']]
    row.extend(image_dict[i[1]['fname']].flatten())
    oot_data.append(row)

oot_data = np.array(oot_data).astype(float)

In [28]:
X_oot = oot_data[:,1:]

y_oot = oot_data[:,0].astype('int32')
y_oot = np.concatenate([1 - y_oot.reshape(-1,1), y_oot.reshape(-1,1)], axis = 1)

In [29]:
print('oot error')
eval_results = car_classifier.evaluate(
    input_fn=lambda:image_processor_eval_input_fn(X_oot, y_oot, 32))
print(eval_results)

oot error
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-15T02:17:57Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-111848
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-15-02:18:05
INFO:tensorflow:Saving dict for global step 111848: eval_accuracy = 0.8368421, global_step = 111848, loss = 149.67972
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 111848: /home/ec2-user/convnet_model/model.ckpt-111848
{'eval_accuracy': 0.8368421, 'loss': 149.67972, 'global_step': 111848}


In [30]:
predicted_classes = car_classifier.predict(
    input_fn=lambda:image_processor_eval_input_fn(testing_image_processor, X_oot, y_oot, 32))

testing_analysis = []

for pred, label in zip(predicted_classes, y_oot):
    hold = [pred['probabilities'][1], pred['classes'], label[1], pred['logits'][0], pred['logits'][1]]
    testing_analysis.append(hold)
    
testing_analysis = pd.DataFrame(testing_analysis, columns = ['prob', 'class', 'label', 'logit_0', 'logit_1'])
testing_analysis['logit_1_decile'] = pd.qcut(testing_analysis.logit_1, 10, labels = False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-111848
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [31]:
testing_analysis.groupby(['logit_1_decile', 'label'])['class'].value_counts()

logit_1_decile  label  class
0               1      0        38
1               1      0        24
                       1        14
2               1      1        38
3               1      1        38
4               1      1        38
5               1      1        38
6               1      1        38
7               1      1        38
8               1      1        38
9               1      1        38
Name: class, dtype: int64

In [ ]:
for i in testing_analysis.loc[testing_analysis.logit_1_decile == 0].index[100:120]:
    plt.figure()
    imshow(Image.fromarray((X_oot[i,:].reshape((200,200))).astype('uint8'), mode = 'L'))

Train and Evaluate

In [9]:
train_spec = tf.estimator.TrainSpec(
     input_fn=lambda:image_processor_train_input_fn(
         training_image_processor, X_train, y_train, 32), 
     max_steps=4100)


eval_spec = tf.estimator.EvalSpec(
    input_fn=lambda:image_processor_eval_input_fn(X_train, y_train, 32)
)

In [11]:
evaluate_results = tf.estimator.train_and_evaluate(car_classifier, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Skipping training since max_steps has already saved.


Export to SavedModel

In [24]:
def serving_input_receiver_fn():
    inputs = {
        'input' : tf.placeholder(tf.float32, [None, 200, 200, 1]),
    }
    
    inputs['input'] = tf.divide(inputs['input'], 255.0)
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)


In [26]:
car_classifier.export_savedmodel('/home/ec2-user/convnet_saved_model/', serving_input_receiver_fn,
                            strip_default_attrs=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-4100
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/ec2-user/convnet_saved_model/temp-b'1553120111'/saved_model.pb


b'/home/ec2-user/convnet_saved_model/1553120111'